In [ ]:
!python -m pip install shioaji

In [8]:
from platform import python_version
print(python_version())


3.7.4


In [3]:
import shioaji as sj

In [4]:
# ---- Formal Login
api = sj.Shioaji()
accounts = api.login(
    "J120420374", 
    "celia5818"
)

api.activate_ca(
    #ca_path="C:/ekey/551/J120420374/S/Sinopac.pfx",
    ca_path="c:/eleader/ca/Sinopac.pfx",
    ca_passwd="J120420374",
    person_id="J120420374",
)


Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


True

In [2]:
# --- 


True

In [8]:
accounts = api.list_accounts()
print(accounts)


[FutureAccount(person_id='J120420374', broker_id='F002000', account_id='1641626', username='黃瑞柳'), Account(account_type=<AccountType.H: 'H'>, person_id='J120420374', broker_id='9A97', account_id='0034119', username='黃瑞柳'), StockAccount(person_id='J120420374', broker_id='9A97', account_id='0296637', signed=True, username='黃瑞柳\u3000\u3000')]


In [5]:
# Stock default account
print(api.stock_account)

# Futures default account
print(api.futopt_account)


person_id='J120420374' broker_id='9A97' account_id='0296637' signed=True username='黃瑞柳\u3000\u3000'
person_id='J120420374' broker_id='F002000' account_id='1641626' signed=True username='黃瑞柳'
Response Code: 0 | Event Code: 12 | Info: solClient.c:12194                    (0000336c) Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/J120420374/1230/045243/255866/114.137.180.28', VPN name 'sinopac', peer address 'hostname '203.66.91.161:80' IP 203. | Event: Session reconnecting
Session Down.
Response Code: 0 | Event Code: 1 | Info: solClientOS.c:4220                   (0000336c) TCP connection failure for fd 1796, error = No route to host (10065) | Event: Session down


--- Quote callback

In [ ]:
from shioaji import TickFOPv1, BidAskFOPv1, Exchange
from collections import defaultdict, deque
from shioaji import TickFOPv1, Exchange

# set context
msg_queue = defaultdict(deque)
api.set_context(msg_queue)

# In order to use context, set bind=True
@api.on_tick_fop_v1(bind=True)
def quote_callback(self, exchange:Exchange, tick:TickFOPv1):
    print(f"Exchange: {exchange}, Tick: {tick}")
    # ---- append quote to message queue
    #self[tick.code].append(tick)
   
@api.on_bidask_fop_v1(bind=True)
def quote_callback(self, exchange:Exchange, bidask:BidAskFOPv1):
    print(f"Exchange: {exchange}, BidAsk: {bidask}")
    # ---- append quote to message queue
    #self[bidask.code].append(bidask)


--- Streaming Market Data

In [3]:
contract_2317 = api.Contracts.Stocks["2317"] ### 鴻海股票
contract_fut_2317_01 = api.Contracts.Futures.DHF['DHF202201'] #鴻海期貨202201 (=DHFA2)


In [ ]:
### 鴻海股票

api.quote.subscribe(
    contract_2317, 
    quote_type = sj.constant.QuoteType.Tick, # or 'tick'
    version = sj.constant.QuoteVersion.v1 # or 'v1'
)

In [ ]:
#鴻海期貨202201 (=DHFA2)

api.quote.subscribe(
    contract=contract_fut_2317_01, 
    quote_type="tick",
    version = sj.constant.QuoteVersion.v1, # or 'v1'
)

'''
api.quote.subscribe(
    contract=contract_fut_2317_01, 
    quote_type = "bidask",
    version = sj.constant.QuoteVersion.v1 # or 'v1'
)
'''

#print(contract_fut_2317_01)

In [ ]:
###### Unsubscribe tick data
api.quote.unsubscribe(contract_2317, quote_type=sj.constant.QuoteType.Tick)
#api.quote.unsubscribe(contract_fut_2317_01, quote_type=sj.constant.QuoteType.Tick)
api.quote.unsubscribe(contract_fut_2317_01, quote_type=sj.constant.QuoteType.BidAsk)

--- Place Order


In [5]:
# 定義place_order、update_order及cancel_order後，要執行什麼動作
# 沒有定義的話，預設是print(stat, msg)
def place_cb(stat, msg):
    print('my_place_callback')
    print(f'stat:{stat}')
    print(f'msg:{msg}')
    api.update_status(api.stock_account) #測試，是否可以在成功發送委託單後，自動更新

api.set_order_callback(place_cb)

In [7]:
# futures order
order_fut_1 = api.Order(
    price=105,
    quantity=1,
    action=sj.constant.Action.Buy,
    price_type=sj.constant.StockPriceType.LMT,
    order_type=sj.constant.FuturesOrderType.ROD, 
    octype=sj.constant.FuturesOCType.Auto,
    account=api.futopt_account

)

#trade = api.place_order(contract_fut_2317_01, order_fut_1, timeout= 5000)
trade = api.place_order(contract_fut_2317_01, order_fut_1)
trade

Trade(contract=Future(code='DHFA2', symbol='DHF202201', name='鴻海期貨01', category='DHF', delivery_month='202201', delivery_date='2022/01/19', underlying_kind='S', underlying_code='2317', unit=1, limit_up=116.5, limit_down=95.4, reference=106.0, update_date='2021/12/29'), order=Order(action=<Action.Buy: 'Buy'>, price=105, quantity=1, account=FutureAccount(person_id='J120420374', broker_id='F002000', account_id='1641626', username='黃瑞柳'), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>), status=OrderStatus(status=<Status.Failed: 'Failed'>))

Response Code: 0 | Event Code: 12 | Info: solClient.c:12194                    (00000a44) Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/J120420374/1229/022808/414560/114.137.81.245', VPN name 'sinopac', peer address 'hostname '203.66.91.161:80' IP 203. | Event: Session reconnecting
Session Down.
Response Code: 0 | Event Code: 1 | Info: solClientOS.c:4220                   (00000a44) TCP connection failure for fd 1840, error = No route to host (10065) | Event: Session down


In [ ]:
# Logout
api.logout()

In [6]:
sj.constant??

Type:        module
String form: <module 'shioaji.constant' from 'C:\\Users\\Perry\\AppData\\Local\\Programs\\Python\\Python37\\lib\\site-packages\\shioaji\\constant.py'>
File:        c:\users\perry\appdata\local\programs\python\python37\lib\site-packages\shioaji\constant.py
Source:     
from enum import Enum

ACTION_BUY = "Buy"
ACTION_SELL = "Sell"

STOCK_PRICE_TYPE_LIMITPRICE = "LMT"
STOCK_PRICE_TYPE_MKT = "MKT"
STOCK_PRICE_TYPE_CLOSE = "Close"
STOCK_PRICE_TYPE_LIMITUP = "LimitUp"
STOCK_PRICE_TYPE_LIMITDOWN = "LimitDown"

STOCK_ORDER_LOT_COMMON = "Common"  # 整股
STOCK_ORDER_LOT_BLOCKTRADE = "BlockTrade"  # 鉅額
STOCK_ORDER_LOT_FIXING = "Fixing"  # 定盤
STOCK_ORDER_LOT_ODD = "Odd"  # 零股
STOCK_ORDER_LOT_INTRADAY_ODD = "IntradayOdd"  # 零股

STOCK_ORDER_TYPE_COMMON = "Common"  # 整股
STOCK_ORDER_TYPE_BLOCKTRADE = "BlockTrade"  # 鉅額
STOCK_ORDER_TYPE_FIXING = "Fixing"  # 定盤
STOCK_ORDER_TYPE_ODD = "Odd"  # 零股

STOCK_ORDER_COND_CASH = "Cash"  # 現股
STOCK_ORDER_COND_NETTING = "Netting"  # 餘額交割
STOCK_O